In [1]:
# %% [markdown]
# # Konwerter Danych Gaia (RA, Dec, Parallax) na Współrzędne Kartezjańskie (XYZ)
#
# Ten notatnik wczytuje dane astronomiczne z katalogu Gaia (rektascensja, deklinacja, paralaksa)
# z pliku CSV, przekształca je na współrzędne kartezjańskie (x, y, z), skaluje je
# do odpowiedniego rozmiaru dla wizualizacji 3D i zapisuje wynik jako plik JSON.

# %%
import pandas as pd
import numpy as np
import json
import os # Dodano do sprawdzania ścieżki pliku

# %% [markdown]
# ## Definicja Funkcji Konwertującej
#
# Poniższa funkcja `convert_gaia_to_xyz` zawiera całą logikę przetwarzania danych.

# %%
def convert_gaia_to_xyz(input_csv_path, output_json_path, scale_factor=100):
    """
    Konwertuje dane Gaia (RA, Dec, Parallax) z pliku CSV na współrzędne kartezjańskie (x, y, z),
    skaluje je i zapisuje jako plik JSON.

    Args:
        input_csv_path (str): Ścieżka do pliku CSV z danymi Gaia.
                               Oczekiwane kolumny: 'ra', 'dec', 'parallax'.
                               Plik powinien zawierać nagłówek w pierwszym wierszu.
        output_json_path (str): Ścieżka do pliku JSON, w którym zostaną zapisane wyniki.
        scale_factor (float): Docelowy promień wizualizacji, używany do skalowania
                               obliczonych odległości w parsekach.
                               Może wymagać dostosowania.
    """
    print(f"Rozpoczynanie konwersji dla pliku: {input_csv_path}")

    # Sprawdzenie, czy plik wejściowy istnieje
    if not os.path.exists(input_csv_path):
        print(f"BŁĄD: Plik wejściowy nie istnieje: {input_csv_path}")
        print("Upewnij się, że plik CSV znajduje się we właściwej lokalizacji.")
        return

    try:
        # Wczytaj dane z CSV używając pandas
        print("Wczytywanie danych z CSV...")
        df = pd.read_csv(input_csv_path, usecols=['ra', 'dec', 'parallax'], header=0)
        print(f"Wczytano {len(df)} wierszy.")

        # --- Filtracja danych (ważne!) ---
        print("Filtrowanie danych...")
        initial_rows = len(df)
        # Usuń wiersze z brakującymi danymi w kluczowych kolumnach
        df.dropna(subset=['ra', 'dec', 'parallax'], inplace=True)
        # Usuń wiersze z nieprawidłową paralaksą (<= 0)
        df = df[df['parallax'] > 0]
        filtered_rows = len(df)
        print(f"Usunięto {initial_rows - filtered_rows} wierszy (NaN lub parallax <= 0). Pozostało {filtered_rows} wierszy.")

        if df.empty:
            print("Brak prawidłowych danych po filtracji. Zatrzymuję przetwarzanie.")
            return

        # --- Konwersja na radiany ---
        print("Konwersja kątów na radiany...")
        ra_rad = np.radians(df['ra'])
        dec_rad = np.radians(df['dec'])

        # --- Obliczanie odległości (r) ---
        print("Obliczanie odległości z paralaksy...")
        # Paralaksę (parallax) podaje się w milisekundach łuku (mas)
        # Odległość w parsekach (pc) = 1000 / paralaksa (mas)
        distance_pc = 1000.0 / df['parallax'] # Wynik w parsekach

        # --- Konwersja na współrzędne kartezjańskie (x, y, z) ---
        print("Konwersja na współrzędne kartezjańskie (x, y, z)...")
        # Standardowe wzory transformacji sferycznej na kartezjańską
        x = distance_pc * np.cos(dec_rad) * np.cos(ra_rad)
        y = distance_pc * np.cos(dec_rad) * np.sin(ra_rad)
        z = distance_pc * np.sin(dec_rad)

        # --- Skalowanie ---
        print("Skalowanie współrzędnych...")
        max_distance = distance_pc.max()
        min_distance = distance_pc.min()
        print(f"Minimalna obliczona odległość: {min_distance:.2f} pc")
        print(f"Maksymalna obliczona odległość: {max_distance:.2f} pc")

        # Skalowanie, aby najdalsza gwiazda była mniej więcej w odległości 'scale_factor' od centrum
        if max_distance > 0:
             scaling_value = max_distance / scale_factor
        else:
             scaling_value = 1.0

        if scaling_value == 0: # Zabezpieczenie
            scaling_value = 1.0

        print(f"Używany współczynnik skalowania (dzielnik odległości): {scaling_value:.2f} (dla docelowego promienia {scale_factor})")

        x_scaled = x / scaling_value
        y_scaled = y / scaling_value
        z_scaled = z / scaling_value

        # --- Przygotowanie danych wyjściowych ---
        print("Przygotowywanie danych do zapisu...")
        output_data = list(zip(x_scaled.tolist(), y_scaled.tolist(), z_scaled.tolist()))

        # --- Zapis do pliku JSON ---
        print(f"Zapisywanie {len(output_data)} przetworzonych współrzędnych do: {output_json_path}")
        with open(output_json_path, 'w') as f:
            json.dump(output_data, f, indent=None) # Zapis bez wcięć dla mniejszego rozmiaru pliku

        print(f"\nKonwersja zakończona sukcesem! Wyniki zapisano w {output_json_path}")

    except FileNotFoundError:
         # Ten błąd jest już obsługiwany na początku funkcji
         pass
    except KeyError as e:
        print(f"BŁĄD: Brak oczekiwanej kolumny w pliku CSV: {e}.")
        print("Sprawdź, czy nagłówek w pliku CSV zawiera kolumny 'ra', 'dec' i 'parallax'.")
    except Exception as e:
        print(f"Wystąpił nieoczekiwany błąd podczas przetwarzania: {e}")

# %% [markdown]
# ## Uruchomienie Konwersji
#
# W tej komórce definiujemy ścieżki do plików wejściowego (CSV) i wyjściowego (JSON)
# oraz docelowy promień wizualizacji, a następnie wywołujemy funkcję konwertującą.
#
# **Pamiętaj, aby dostosować `input_file` do nazwy Twojego pliku CSV!**

# %%
# --- Konfiguracja ---
# !!! ZMIEŃ TĘ NAZWĘ, jeśli Twój plik CSV nazywa się inaczej !!!
input_file = 'artur_top100000_najjasniejszych_v1-result.csv'
output_file = 'star_positions.json' # Nazwa pliku JSON do wygenerowania

# Docelowy promień wizualizacji (np. 100 jednostek w Three.js)
# Wpływa na skalowanie współrzędnych.
TARGET_VISUALIZATION_RADIUS = 100.0

# --- Wywołanie funkcji ---
convert_gaia_to_xyz(input_file, output_file, scale_factor=TARGET_VISUALIZATION_RADIUS)

# %% [markdown]
# --- Koniec Notatnika ---


Rozpoczynanie konwersji dla pliku: artur_top100000_najjasniejszych_v1-result.csv
Wczytywanie danych z CSV...
Wczytano 100000 wierszy.
Filtrowanie danych...
Usunięto 0 wierszy (NaN lub parallax <= 0). Pozostało 100000 wierszy.
Konwersja kątów na radiany...
Obliczanie odległości z paralaksy...
Konwersja na współrzędne kartezjańskie (x, y, z)...
Skalowanie współrzędnych...
Minimalna obliczona odległość: 1.83 pc
Maksymalna obliczona odległość: 7175.18 pc
Używany współczynnik skalowania (dzielnik odległości): 71.75 (dla docelowego promienia 100.0)
Przygotowywanie danych do zapisu...
Zapisywanie 100000 przetworzonych współrzędnych do: star_positions.json

Konwersja zakończona sukcesem! Wyniki zapisano w star_positions.json
